In [18]:
# Import necessary libraries for model tokenization, sequence classification, tensor computations, and numerical operations
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

# Define the name of the pre-trained model to be used for sequence classification
model_name = "Volavion/bert-base-multilingual-uncased-Temperature-CLS"

# Load the tokenizer and model associated with the specified model name
# The tokenizer processes text into numerical input for the model, while the model performs sequence classification
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Retrieve the maximum token length supported by the tokenizer, useful for defining input constraints
max_token_length = tokenizer.model_max_length

# Define the input text to be analyzed by the model
input_text = """You are an AI storyteller tasked with crafting an innovative and thought-provoking narrative. Your story should explore the intersection of technology, humanity, and the environment in a near-future world. Use vivid imagery, compelling characters, and unexpected twists to engage the reader.

Scenario:
In the year 2045, Earth faces a tipping point due to climate change. Amidst this chaos, a young scientist discovers an ancient technology buried deep within the Arctic ice. This technology has the potential to either save the planet or accelerate its demise. The scientist must navigate corporate greed, political intrigue, and personal dilemmas to decide the fate of humanity.
"""

# Tokenize the input text:
# - `add_special_tokens=True` ensures inclusion of special tokens like [CLS] and [SEP]
# - `max_length=512` limits input size to prevent exceeding model constraints
# - `pad_to_max_length=True` pads shorter inputs to the maximum length
# - `return_attention_mask=True` generates an attention mask for the model
# - `return_tensors="pt"` outputs PyTorch tensors for model compatibility
encoded_dict = tokenizer.encode_plus(
        input_text,  
        add_special_tokens=True,  
        max_length=512, 
        pad_to_max_length=True,
        return_attention_mask=True,  
        return_tensors="pt",  # Return pytorch tensors.
)

# Extract the token IDs and attention mask from the tokenized dictionary
input_ids = encoded_dict["input_ids"]
attention_mask = encoded_dict["attention_mask"]

# Determine the appropriate computation device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the selected device for computation
model.to(device)

# Set the model to evaluation mode (disables training-specific behaviors like dropout)
model.eval()

# Transfer input tensors to the same device as the model
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Perform inference without gradient computation to save memory and speed up evaluation
with torch.no_grad():
    # Obtain the model's output logits (raw predictions)
    outputs = model(input_ids, attention_mask=attention_mask)

# Convert logits to a NumPy array for easier manipulation
logits = outputs.logits.cpu().numpy()

# Apply softmax transformation to logits to compute probabilities for each class
# Subtracting the max value (logits stability trick) prevents overflow during exponentiation
probabilities = np.exp(logits - np.max(logits, axis=1, keepdims=True))  
probabilities = probabilities / np.sum(probabilities, axis=1, keepdims=True)

# Map class indices to their corresponding abilities
ability_mapping = {0: "Causal Reasoning", 1: "Creativity", 2: "In-Context Learning", 
                   3: "Instruction Following", 4: "Machine Translation", 5: "Summarization"}

# Format the probabilities as percentages for each class and construct a readable output string
formatted_output = "\n".join(
    [f"{category}: {prob*100:.2f}%" for prob, category in zip(probabilities[0], ability_mapping.values())]
)

# Display the classification probabilities for each ability
print("Ability Classification Probabilities:")
print(formatted_output)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Ability Classification Probabilities:
Causal Reasoning: 17.90%
Creativity: 22.17%
In-Context Learning: 16.48%
Instruction Following: 12.79%
Machine Translation: 13.15%
Summarization: 17.51%
